In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/meta-arabic-data/laws_ar.index.meta
/kaggle/input/bilangual-data/algerian_legal(joconstitutionpenalecivilcommercefamille) embedder_ dangvantuan-sentence-camembert-large.faiss
/kaggle/input/bilangual-data/laws_ar.index
/kaggle/input/bilangual-data/algerian_legal(joconstitutionpenalecivilcommercefamille) embedder_ dangvantuan-sentence-camembert-large_docs.json


In [2]:
!pip install -q transformers torch sentence-transformers faiss-cpu accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 94.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 32.6 MB/s eta 0:00:00:00:0100:01


In [3]:
import json
import torch
import faiss
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
from typing import List, Dict
import warnings
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully!")
print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")

2026-01-01 21:41:18.730532: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767303679.179298      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767303679.298368      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767303680.351252      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767303680.351315      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767303680.351317      55 computation_placer.cc:177] computation placer alr

✅ All libraries imported successfully!
✅ PyTorch version: 2.8.0+cu126
✅ CUDA available: True


In [4]:

# French files
FRENCH_INDEX_PATH = "/kaggle/input/bilangual-data/algerian_legal(joconstitutionpenalecivilcommercefamille) embedder_ dangvantuan-sentence-camembert-large.faiss"
FRENCH_DOCS_PATH = "/kaggle/input/bilangual-data/algerian_legal(joconstitutionpenalecivilcommercefamille) embedder_ dangvantuan-sentence-camembert-large_docs.json"

# Arabic files
ARABIC_INDEX_PATH = "/kaggle/input/bilangual-data/laws_ar.index"

# Models
FRENCH_EMBEDDING_MODEL = "dangvantuan/sentence-camembert-large"
ARABIC_EMBEDDING_MODEL = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
FRENCH_LLM = "bofenghuang/vigogne-2-7b-chat"
ARABIC_LLM = "Qwen/Qwen2.5-7B-Instruct"

USE_4BIT_QUANTIZATION = True
TOP_K_RETRIEVAL = 3

print("✅ Configuration set!")

✅ Configuration set!


In [5]:
def detect_script(text: str) -> str:
    """Detect if text is Arabic or French/Latin script"""
    arabic_chars = sum(1 for c in text if '\u0600' <= c <= '\u06FF')
    total_chars = sum(1 for c in text if c.isalpha())
    
    if total_chars == 0:
        return "french"
    
    arabic_ratio = arabic_chars / total_chars
    return "arabic" if arabic_ratio > 0.5 else "french"

def detect_response_language(text: str) -> str:
    """Detect requested response language from query"""
    text_lower = text.lower()
    
    # Check for explicit language requests
    arabic_requests = ["en arabe", "in arabic", "بالعربية", "answer in arabic", 
                      "répondre en arabe", "أجب بالعربية"]
    french_requests = ["en français", "in french", "بالفرنسية", "answer in french", 
                      "répondre en français", "أجب بالفرنسية"]
    
    for req in arabic_requests:
        if req in text_lower:
            return "arabic"
    
    for req in french_requests:
        if req in text_lower:
            return "french"
    
    # Default: same as query language
    return detect_script(text)

print("✅ Language detection functions defined!")



✅ Language detection functions defined!


In [6]:
print("🔄 Loading embedding models...")

french_embedder = SentenceTransformer(FRENCH_EMBEDDING_MODEL)
print(f"✅ French embedder loaded: {FRENCH_EMBEDDING_MODEL}")

arabic_embedder = SentenceTransformer(ARABIC_EMBEDDING_MODEL)
print(f"✅ Arabic embedder loaded: {ARABIC_EMBEDDING_MODEL}")


🔄 Loading embedding models...


config.json:   0%|          | 0.00/683 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/400 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/809k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Arabic embedder loaded: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2


In [7]:
import pickle

print("🔄 Loading FAISS indices and documents...")

# French
french_index = faiss.read_index(FRENCH_INDEX_PATH)
with open(FRENCH_DOCS_PATH, 'r', encoding='utf-8') as f:
    french_docs = json.load(f)
print(f"✅ French index: {french_index.ntotal} vectors, {len(french_docs)} documents")

# Arabic
arabic_index = faiss.read_index(ARABIC_INDEX_PATH)
with open("/kaggle/input/meta-arabic-data/laws_ar.index.meta", "rb") as f:
    arabic_meta = pickle.load(f)

arabic_docs = arabic_meta["chunks"]
print(f"✅ Arabic index: {arabic_index.ntotal} vectors, {len(arabic_docs)} documents")


🔄 Loading FAISS indices and documents...
✅ French index: 18722 vectors, 18722 documents
✅ Arabic index: 12758 vectors, 12758 documents


In [8]:
print("🔄 Loading French LLM...")

french_tokenizer = AutoTokenizer.from_pretrained(FRENCH_LLM)

if USE_4BIT_QUANTIZATION and torch.cuda.is_available():
    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )
    french_llm = AutoModelForCausalLM.from_pretrained(
        FRENCH_LLM,
        quantization_config=quant_config,
        device_map="auto",
        trust_remote_code=True
    )
else:
    french_llm = AutoModelForCausalLM.from_pretrained(
        FRENCH_LLM,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto" if torch.cuda.is_available() else None,
        trust_remote_code=True
    )

print(f"✅ French LLM loaded: {FRENCH_LLM}")

🔄 Loading French LLM...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

...

pytorch_model-00006-of-00007.bin:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

✅ French LLM loaded: bofenghuang/vigogne-2-7b-chat


In [9]:
print("🔄 Loading Arabic LLM...")

arabic_tokenizer = AutoTokenizer.from_pretrained(ARABIC_LLM)

if USE_4BIT_QUANTIZATION and torch.cuda.is_available():
    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )
    arabic_llm = AutoModelForCausalLM.from_pretrained(
        ARABIC_LLM,
        quantization_config=quant_config,
        device_map="auto",
        trust_remote_code=True
    )
else:
    arabic_llm = AutoModelForCausalLM.from_pretrained(
        ARABIC_LLM,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto" if torch.cuda.is_available() else None,
        trust_remote_code=True
    )

print(f"✅ Arabic LLM loaded: {ARABIC_LLM}")

🔄 Loading Arabic LLM...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

✅ Arabic LLM loaded: Qwen/Qwen2.5-7B-Instruct


In [10]:
def retrieve_documents(query: str, response_lang: str, top_k: int = 3) -> List[Dict]:
    """
    Retrieve relevant documents based on RESPONSE language
    
    KEY CHANGE: Now uses response_lang instead of query_lang
    This ensures we search in the language we want to answer in
    """
    
    # Select appropriate embedder, index, and documents based on RESPONSE language
    if response_lang == "arabic":
        embedder = arabic_embedder
        index = arabic_index
        docs = arabic_docs
        print(f"   → Retrieving from ARABIC documents (response lang: {response_lang})")
    else:
        embedder = french_embedder
        index = french_index
        docs = french_docs
        print(f"   → Retrieving from FRENCH documents (response lang: {response_lang})")
    
    # Encode query (the embedder will handle cross-lingual queries if it's multilingual)
    query_embedding = embedder.encode(
        [query],
        convert_to_numpy=True,
        normalize_embeddings=True
    )
    
    # Search
    scores, indices = index.search(query_embedding.astype('float32'), top_k)
    
    # Return documents with scores
    results = []
    for idx, score in zip(indices[0], scores[0]):
        if idx < len(docs) and idx != -1:
            doc = docs[idx].copy()
            doc['score'] = float(score)
            results.append(doc)
    
    return results

print("✅ Retrieval function defined (MODIFIED - uses response language)!")

✅ Retrieval function defined (MODIFIED - uses response language)!


In [11]:
def build_context(docs: List[Dict], response_lang: str) -> str:
    """Build context from retrieved documents based on response language"""
    context_parts = []
    
    for doc in docs:
        if response_lang == "arabic":
            titre = doc.get('titre', '')
            texte = doc.get('texte', '')
            ctx = f"{titre}\n{texte}"
        else:
            doc_type = doc.get('source_document_type', '').upper()
            header = doc.get('header', '')
            content = doc.get('content', '')
            ctx = f"[{doc_type}] {header}\n{content}"
        
        context_parts.append(ctx)
    
    return "\n\n".join(context_parts)

def generate_french_answer(question: str, context: str) -> str:
    """Generate answer in French"""
    prompt = f"""<|system|>: Tu es un assistant juridique expert en droit algérien. Réponds de manière précise, professionnelle et factuelle en te basant strictement sur le contexte fourni.
<|user|>: Contexte juridique:
{context}

Question: {question}
<|assistant|>:"""
    
    inputs = french_tokenizer(prompt, return_tensors="pt")
    if torch.cuda.is_available():
        inputs = {k: v.to("cuda") for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = french_llm.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.7,
            top_p=0.95,
            do_sample=True,
            pad_token_id=french_tokenizer.eos_token_id,
            repetition_penalty=1.1
        )
    
    response = french_tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    if "<|assistant|>:" in response:
        return response.split("<|assistant|>:")[-1].strip()
    else:
        return response[len(prompt):].strip()

def generate_arabic_answer(question: str, context: str) -> str:
    """Generate answer in Arabic"""
    prompt = f"""You are an Algerian Legal Expert. Answer based ONLY on the context below.
    
Context:
{context}

Question: {question}

Answer (in Arabic, comprehensive):"""
    
    inputs = arabic_tokenizer(prompt, return_tensors="pt")
    if torch.cuda.is_available():
        inputs = {k: v.to("cuda") for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = arabic_llm.generate(
            **inputs,
            max_new_tokens=1024,
            temperature=0.3,
            repetition_penalty=1.1
        )
    
    response = arabic_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.split("Answer (in Arabic, comprehensive):")[-1].strip()

print("✅ Generation functions defined!")

✅ Generation functions defined!


In [12]:
def bilingual_query(question: str, top_k: int = 3, show_sources: bool = True) -> Dict:
    """
    Process a bilingual query and return answer
    
    KEY CHANGE: Now retrieves documents based on RESPONSE language
    """
    
    print(f"\n{'='*70}")
    print(f"Question: {question}")
    print('='*70)
    
    # Detect languages
    query_lang = detect_script(question)
    response_lang = detect_response_language(question)
    
    print(f"\n🔍 Query Language: {query_lang.upper()}")
    print(f"💬 Response Language: {response_lang.upper()}")
    
    # KEY CHANGE: Retrieve documents based on RESPONSE language (not query language)
    print(f"\n📚 Retrieving documents based on RESPONSE language ({response_lang})...")
    docs = retrieve_documents(question, response_lang, top_k)
    
    # Generate answer in response language
    print(f"🤖 Generating answer in {response_lang}...")
    context = build_context(docs, response_lang)
    
    if response_lang == "arabic":
        answer = generate_arabic_answer(question, context)
    else:
        answer = generate_french_answer(question, context)
    
    # Display results
    print(f"\n📝 Answer:\n{answer}\n")
    
    if show_sources:
        print("📚 Sources:")
        for i, doc in enumerate(docs, 1):
            if response_lang == "arabic":
                titre = doc.get('titre', 'N/A')[:80]
                score = doc.get('score', 0)
                print(f"\n  [{i}] {titre}")
                print(f"      Relevance: {score:.4f}")
            else:
                doc_type = doc.get('source_document_type', '').upper()
                header = doc.get('header', 'N/A')[:80]
                score = doc.get('score', 0)
                print(f"\n  [{i}] {doc_type} - {header}")
                print(f"      Relevance: {score:.4f}")
    
    return {
        "answer": answer,
        "sources": docs,
        "query_language": query_lang,
        "response_language": response_lang
    }

print("✅ Main query function defined (MODIFIED - uses response language)!")

✅ Main query function defined (MODIFIED - uses response language)!


In [13]:
result1 = bilingual_query("Qu'est-ce que l'Algérie selon la constitution?")



Question: Qu'est-ce que l'Algérie selon la constitution?

🔍 Query Language: FRENCH
💬 Response Language: FRENCH

📚 Retrieving documents based on RESPONSE language (french)...
   → Retrieving from FRENCH documents (response lang: french)
🤖 Generating answer in french...

📝 Answer:
Selon la Constitution de l'Algérie, l'Algérie est une République démocratique et populaire, ainsi qu'une et indivisible. La capitale de cette république est Alger.

📚 Sources:

  [1] CONSTITUTION - Title I - Chapter PREMIER - Article 1er
      Relevance: 0.4948

  [2] CONSTITUTION - Title I - Chapter PREMIER - Article 5
      Relevance: 0.4773

  [3] CODE_CIVIL - EUXIEME DE LA REPRESENTATION LEGALE - LE LIVRET DE FAMILLE ET LES FICHES D'ETAT 
      Relevance: 0.4696


In [17]:
result_comparaison = bilingual_query("Qu'est-ce que l'Algérie selon la constitution? en arabe")


Question: Qu'est-ce que l'Algérie selon la constitution? en arabe

🔍 Query Language: FRENCH
💬 Response Language: ARABIC

📚 Retrieving documents based on RESPONSE language (arabic)...
   → Retrieving from ARABIC documents (response lang: arabic)
🤖 Generating answer in arabic...


OutOfMemoryError: CUDA out of memory. Tried to allocate 8.13 GiB. GPU 0 has a total capacity of 14.74 GiB of which 6.61 GiB is free. Process 4461 has 8.12 GiB memory in use. Of the allocated memory 7.68 GiB is allocated by PyTorch, and 328.87 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [15]:
result2 = bilingual_query("ما هي عقوبة اختلاس الأموال العمومية؟")



Question: ما هي عقوبة اختلاس الأموال العمومية؟

🔍 Query Language: ARABIC
💬 Response Language: ARABIC

📚 Retrieving documents based on RESPONSE language (arabic)...
   → Retrieving from ARABIC documents (response lang: arabic)
🤖 Generating answer in arabic...

📝 Answer:
السياق الذي تم تقديمكم به لا يحتوي على معلومات حول عقوبة اختلاس الأموال العمومية. هذا السياق يتحدث عن تقسيم التركة عند وجود ذوي فروض وإجراءات تنفيذ العقود. لذا، لن يكون من المناسب الإجابة بناءً على هذا السياق. ومع ذلك، يمكنني توفير المعلومات القانونية العامة المتعلقة بعقوبات اختلاس الأموال العمومية في الجزائر:

اختلاس الأموال العمومية يعتبر جريمة خطيرة في القانون الجزائري، وفقا للفصل 408 من قانون العقوبات الجزائري. العقوبة قد تتراوح بين السجن لمدة تصل إلى عشر سنوات وغرامة مالية كبيرة. في بعض الحالات الخطيرة، قد يصل السجن إلى أكثر من عشر سنوات. كما قد يتم استرداد الأموال المختلسة وإعادة توزيعها وفقا للقوانين ذات الصلة. 

من الجدير بالذكر أن هذه المعلومات عامة وقد يتغير القانون حسب السياق القانوني والظروف الخاصة لكل حالة.

In [18]:
result3 = bilingual_query("Quels sont les droits fondamentaux? Réponds en arabe.")



Question: Quels sont les droits fondamentaux? Réponds en arabe.

🔍 Query Language: FRENCH
💬 Response Language: ARABIC

📚 Retrieving documents based on RESPONSE language (arabic)...
   → Retrieving from ARABIC documents (response lang: arabic)
🤖 Generating answer in arabic...

📝 Answer:
في القانون الجزائري، لا يوجد تعبير محدد للحقوق fundamentales كما في بعض其他国家不能回答这个问题，请基于提供的上下文进行回答。
在阿尔及利亚法律中，并没有明确使用“基本权利”（حقوق fundamentales）这一术语。然而，根据《阿尔及利亚民法典》和《阿尔及利亚宪法》，一些基本原则和权利被认为是基础性的。例如，所有权、继承权以及个人身份等权利被视为基本的财产权利和个人自由。此外，《阿尔及利亚宪法》第29条确认了所有公民享有生命、尊严和自由的权利，这是最基本的人权之一。

请注意，上述内容是基于提供的有限上下文信息进行的解释，可能无法涵盖所有相关方面。具体的权利和原则应参考完整的法律文本以获得准确的信息。 在此情况下，我们讨论的是与遗产分配和占有相关的法律规定，而不是广泛意义上的基本权利。 因此，对于“基本权利”的定义和范围需要进一步澄清或提供更广泛的背景信息。在此范围内，我们只能提到一些与财产和继承有关的基本原则。 如果您需要更详细的信息，建议查阅具体的法律文件或咨询专业的法律顾问。 

答案（仅基于给定的上下文）：
في هذا السياق، لا يمكن تحديد حقوق fundamentales بشكل واضح. ومع ذلك، يتم تقسيم التركة وفقًا لنصب الفروض، حيث يتم تقسيمها بنسب أنصاب الفروض عند زيادة هذه النصابات على أصل المسألة. بالإضافة إلى ذلك، يتم نقل المل

In [19]:
result4 = bilingual_query("Quels sont les droits fondamentaux?")



Question: Quels sont les droits fondamentaux?

🔍 Query Language: FRENCH
💬 Response Language: FRENCH

📚 Retrieving documents based on RESPONSE language (french)...
   → Retrieving from FRENCH documents (response lang: french)
🤖 Generating answer in french...

📝 Answer:
Dans ce contexte juridique, les droits fondamentaux sont mentionnés dans plusieurs articles de la Constitution, notamment dans l'article 34 et 35. Voici une liste non exhaustive des droits fondamentaux évoqués dans ces articles :

1. Droits fondamentaux énoncés dans l'article 34 (loi) :
a. Droit à la vie privée et familiale, y compris la protection des données personnelles (loi)
b. Droit à la liberté de conscience et de religion (loi)
c. Droit à la liberté d'expression, d'opinion et de presse (loi)
d. Droit à la liberté de réunion, de manifestation pacifique et de pétition (loi)
e. Droit à la liberté de circulation et de séjour (loi)
f. Droit à l'égalité devant la justice, y compris le droit à un procès équitable et rapi